## Imports
Execute the cell below to import all packages needed.

In [1]:
# System functionality
import sys
sys.path.append('..')

import glob
import os
import numpy as np 
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as font_manager
import seaborn as sns
import graphviz
import math as math
import json
import numpy
import cv2
import scipy.stats as stats
import matplotlib.image as mpimg
import sklearn.feature_selection as feature_selection
from skimage import data, io, filters
from numpy import linspace
from numpy.core.umath import pi
from matplotlib.ticker import FuncFormatter
from matplotlib.font_manager import FontProperties
from emoji_extractor.extract import Extractor
from emojipedia import Emojipedia
from skimage import io
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import chi2_contingency
from scipy.interpolate import spline

## Load csv into pandas dataframe

In [2]:
path__tweets_csv = '/home/stelios/Desktop/Honours Project/Samples/tweet_data/tweets_ran_all.csv'
path_users_csv = '/home/stelios/Desktop/Honours Project/Samples/user_data/ran.csv'

ran_join_df = pd.read_csv(path__tweets_csv,index_col=None, header=0, low_memory = False).applymap(str).join(pd.read_csv(path_users_csv,index_col=None, header=0, low_memory = False).applymap(str).set_index('user_id'), on='user_id')

## Extract, Count & Sort Emojis for all

In [3]:
extract = Extractor()

count = extract.count_all_emoji(ran_join_df['text'])

# Create panda dataframe to store emojis with counts
emojis_count_df = pd.DataFrame(columns = ['Emoji','Count'])
for emoji in count:
    emojis_count_df = emojis_count_df.append({'Emoji' : emoji , 'Count' : count[emoji]} , ignore_index=True)

# Sort dataframe by count
emojis_count_df = emojis_count_df.sort_values(by=['Count'])
emojis_count_df.tail(20)

,Emoji,Count
99,💗,33495
135,🙄,36686
21,💙,38404
28,🙏,38783
151,♥,38865
2,👍,39711
36,😘,42422
26,👏,46411
50,🔥,50812
9,🤔,51281


## TABLE

In [4]:
# Load users
ran_users_df = pd.read_csv(path_users_csv,index_col=None, header=0, low_memory = False).applymap(str)

# Calculate lengths
length_emojis = len(emojis_count_df['Emoji'].values)
print(length_emojis)
length_users = len(ran_users_df['user_id'])

# Create zeros matrix
zeros_matrix= numpy.zeros(shape=(length_users,length_emojis))

# Fill pandas dataframe with zeros
emojis_count_create_df = pd.DataFrame(zeros_matrix,columns = emojis_count_df['Emoji'].values)
emojis_count_create_df

# Initialise extractor
extract = Extractor()

# Add the frequencies of emojis each emoji by User
for enum,user in enumerate (ran_users_df['user_id']):
    user_posts= "".join(ran_join_df.loc[ran_join_df['user_id'] == user, 'text'])
    text_count_emoji = extract.count_emoji(user_posts, check_first=True)
    if(len(text_count_emoji)==0):
        pass
    else:
        for emoji in text_count_emoji:
            emojis_count_create_df.at[enum, emoji] = text_count_emoji[emoji]

# Describe
emojis_count_create_df.info(memory_usage='deep')

2348
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Columns: 2361 entries, 👨🏿‍🔬 to 🇧🇼
dtypes: float64(2361)
memory usage: 180.1 MB


In [5]:
# Fill n/a with 0.0
emojis_count_create_df = emojis_count_create_df.fillna(0.0)

# Store table
emojis_count_create_df.to_csv("/home/stelios/Desktop/Honours Project/Samples/per_user_emoji_counts/random_per_user_emoji_counts.csv",encoding='utf-8', index=False)

In [6]:
# Testing
emojis_count_create_df['😂']

0         0.0
1         0.0
2         8.0
3         0.0
4       145.0
5         0.0
6         0.0
7         1.0
8         0.0
9        36.0
10      251.0
11        3.0
12        0.0
13        0.0
14        8.0
15        0.0
16        0.0
17        0.0
18        1.0
19        1.0
20        0.0
21       18.0
22        5.0
23        2.0
24        9.0
25        0.0
26        0.0
27        0.0
28        0.0
29        2.0
        ...  
9970      0.0
9971      0.0
9972      0.0
9973      0.0
9974      0.0
9975      0.0
9976      0.0
9977      0.0
9978      0.0
9979      0.0
9980      0.0
9981     17.0
9982      0.0
9983     25.0
9984      0.0
9985     12.0
9986      0.0
9987      1.0
9988      0.0
9989      1.0
9990      0.0
9991     90.0
9992      0.0
9993      7.0
9994      0.0
9995      0.0
9996      4.0
9997      0.0
9998      0.0
9999      0.0
Name: 😂, Length: 10000, dtype: float64